In [ ]:
import os

os.mkdir('/content/datasets/')
os.mkdir('/content/datasets/input')

!unzip /content/archive_licenses.zip -d /content/datasets/input/

Archive:  /content/archive_licenses.zip
  inflating: /content/datasets/input/annotations/Cars0.xml  
  inflating: /content/datasets/input/annotations/Cars1.xml  
  inflating: /content/datasets/input/annotations/Cars10.xml  
  inflating: /content/datasets/input/annotations/Cars100.xml  
  inflating: /content/datasets/input/annotations/Cars101.xml  
  inflating: /content/datasets/input/annotations/Cars102.xml  
  inflating: /content/datasets/input/annotations/Cars103.xml  
  inflating: /content/datasets/input/annotations/Cars104.xml  
  inflating: /content/datasets/input/annotations/Cars105.xml  
  inflating: /content/datasets/input/annotations/Cars106.xml  
  inflating: /content/datasets/input/annotations/Cars107.xml  
  inflating: /content/datasets/input/annotations/Cars108.xml  
  inflating: /content/datasets/input/annotations/Cars109.xml  
  inflating: /content/datasets/input/annotations/Cars11.xml  
  inflating: /content/datasets/input/annotations/Cars110.xml  
  inflating: /content

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
!pip install -U -r yolov5/requirements.txt  # install dependencies

Cloning into 'yolov5'...
remote: Enumerating objects: 9848, done.
remote: Total 9848 (delta 0), reused 0 (delta 0), pack-reused 9848
Receiving objects: 100% (9848/9848), 10.23 MiB | 22.42 MiB/s, done.
Resolving deltas: 100% (6818/6818), done.
     |████████████████████████████████| 10.3 MB 5.7 MB/s 
     |████████████████████████████████| 15.7 MB 60 kB/s 
     |████████████████████████████████| 60.3 MB 39 kB/s 
     |████████████████████████████████| 3.1 MB 30.4 MB/s 
     |████████████████████████████████| 596 kB 37.7 MB/s 
     |████████████████████████████████| 62 kB 823 kB/s 
     |████████████████████████████████| 28.5 MB 55 kB/s 
     |██████████████████████████████▎ | 834.1 MB 1.4 MB/s eta 0:00:33tcmalloc: large alloc 1147494400 bytes == 0x5566746f2000 @  0x7f58188d7615 0x55663adcc4cc 0x55663aeac47a 0x55663adcf2ed 0x55663aec0e1d 0x55663ae42e99 0x55663ae3d9ee 0x55663add0bda 0x55663ae42d00 0x55663ae3d9ee 0x55663add0bda 0x55663ae3f737 0x55663aec1c66 0x55663ae3edaf 0x55663aec1c66 0x

In [ ]:
os.rename('/content/datasets/input/annotations/', '/content/datasets/input/labels/')

In [ ]:
import numpy
import os

img_DIR = '/content/datasets/input/images/'
xml_DIR = '/content/datasets/input/labels/'

def get_xml(img):
  return xml_DIR + img[:len(img)-4] + '.xml'

def get_img(xml):
  return img_DIR + img[:len(img)-4] + '.png' 

dir0 = get_xml('Cars1.png')

In [ ]:
import xml.etree.ElementTree as ET

def get_coords(xml_path):
    tree = ET.parse(xml_path)
    for elem in tree.iter(): # we get the dimensions of the image
        if 'size' in elem.tag:
            for attr in list(elem):
                if 'width' in attr.tag:
                      width = int(round(float(attr.text)))
                if 'height' in attr.tag:
                      height = int(round(float(attr.text)))
        if 'object' in elem.tag: # we get the coords
            for attr in list(elem):
                name = attr.text
                for dim in list(attr):
                        if 'xmin' in dim.tag:
                            xmin = int(round(float(dim.text)))
                        if 'ymin' in dim.tag:
                            ymin = int(round(float(dim.text)))
                        if 'xmax' in dim.tag:
                            xmax = int(round(float(dim.text)))
                        if 'ymax' in dim.tag:
                            ymax = int(round(float(dim.text)))
    xc,yc = int((xmin+xmax)/2),int((ymin+ymax)/2) # we get the central coords
    w,h = xmax-xmin,ymax-ymin # we get the dimension of the rectangle
    return [xc/width,yc/height,w/width,h/height] # normalization of the data

In [ ]:
get_coords('/content/datasets/input/labels/Cars108.xml')

[0.49095022624434387,
 0.6966966966966966,
 0.2692307692307692,
 0.0960960960960961]

In [ ]:
for image_name in os.listdir(img_DIR):
  if '.png' in image_name:
    xml_path = get_xml(image_name)
    xc,yc,w,h = get_coords(xml_path)
    f = open(xml_DIR + image_name[:len(image_name)-4] + '.txt', 'w')
    with open(xml_DIR + image_name[:len(image_name)-4] + '.txt', 'w') as f:
        f.write('0' + ' ' + str(xc) + ' ' + str(yc) + ' ' + str(w) + ' ' + str(h) + '\n')
    os.remove(os.path.join(xml_DIR, image_name[:len(image_name)-4]  + '.xml'))

In [ ]:
f = open('/content/yolov5/data/licenses.yaml', 'w')
with open('/content/yolov5/data/licenses.yaml', 'w') as f:
  f.write('path: /content/datasets/input\n')
  f.write('train: train.txt\n')
  f.write('val: test.txt\n')
  f.write('nc: 1\n')
  f.write('names: ["license"]')

In [ ]:
import random, shutil

threshold = int(0.75*len(os.listdir('/content/datasets/input/images/')))
lst_names = [img_name[:len(img_name)-4] for img_name in os.listdir('/content/datasets/input/images/')]
random.shuffle(lst_names)
lst_train = lst_names[:threshold]
lst_test = lst_names[threshold:]

f = open('/content/datasets/input/train.txt', 'w')
with open('/content/datasets/input/train.txt', 'w') as f:
  for image_name in lst_train:
    img_path = os.path.join(img_DIR, image_name + '.png')
    f.write(img_path + '\n')

f = open('/content/datasets/input/test.txt', 'w')
with open('/content/datasets/input/test.txt', 'w') as f:
  for image_name in lst_test:
    img_path = os.path.join(img_DIR, image_name + '.png')
    f.write(img_path + '\n')

In [ ]:
!python yolov5/train.py --img 500 --batch 16 --epochs 300 --data licenses.yaml --weights yolov5s.pt

train: weights=yolov5s.pt, cfg=, data=licenses.yaml, hyp=yolov5/data/hyps/hyp.scratch.yaml, epochs=300, batch_size=16, imgsz=500, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=yolov5/runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: skipping check (not a git repository), for updates see https://github.com/ultralytics/yolov5
YOLOv5 🚀 v6.0-39-g5d4258f torch 1.10.0+cu102 CUDA:0 (Tesla K80, 11441.1875MB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.

In [ ]:
!python yolov5/detect.py --source /content/datasets/input/images/Cars251.png --weights /content/yolov5/runs/train/exp/weights/best.pt --img 500

detect: weights=['/content/yolov5/runs/train/exp/weights/best.pt'], source=/content/datasets/input/images/Cars251.png, imgsz=[500, 500], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.0-39-g5d4258f torch 1.10.0+cu102 CUDA:0 (Tesla K80, 11441.1875MB)

Fusing layers... 
/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /content/datasets/input/im

In [ ]:
from IPython.display import Image

Image('runs/train/exp/val_batch0_pred.jpg')

In [ ]:
from google.colab import files
files.download('/content/yolov5/runs/train/exp/weights/best.pt') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>